In [1]:
#!pip install git+https://github.com/JoaquinAmatRodrigo/skforecast#master --upgrade

# Unit Testing

In [4]:
# Unit test __init__
# ==============================================================================
import sys
sys.path.insert(1, '/home/ximo/Documents/GitHub/skforecast')

import pytest
from pytest import approx
import numpy as np
from skforecast.utils import check_y

def test_check_y_exception_when_y_is_int():
    '''
    '''
    with pytest.raises(Exception):
        check_y(y=10)

In [4]:
# Unit test _create_lags
# ==============================================================================

import sys
sys.path.insert(1, '/home/ximo/Documents/GitHub/skforecast')

import pytest
from pytest import approx
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.linear_model import LinearRegression


def test_create_lags_output():
    '''
    Test matrix of lags is created properly when langs=3 and y=np.arange(10).
    '''
    forecaster = ForecasterAutoreg(LinearRegression(), lags=3)
    results = forecaster._create_lags(y=np.arange(10))
    expected = (np.array([[2., 1., 0.],
                         [3., 2., 1.],
                         [4., 3., 2.],
                         [5., 4., 3.],
                         [6., 5., 4.],
                         [7., 6., 5.],
                         [8., 7., 6.]]),
               np.array([3., 4., 5., 6., 7., 8., 9.]))

    assert (results[0] == expected[0]).all()
    assert (results[1] == expected[1]).all()
    
    
def test_create_lags_exception_when_len_of_y_is_lower_than_maximum_lag():
    '''
    Test exception is raised when lenght of y is lower than maximum lag included
    in the forecaster.
    '''
    forecaster = ForecasterAutoreg(LinearRegression(), lags=10)
    with pytest.raises(Exception):
        forecaster._create_lags(y=np.arange(5))

test_create_lags_output()
test_create_lags_exception_when_len_of_y_is_lower_than_maximum_lag()

In [10]:
# Unit test create_train_X_y
# ==============================================================================
import pytest
from pytest import approx
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.linear_model import LinearRegression


def test_create_train_X_y_output_when_y_is_series_10_and_exog_is_None():
    '''
    Test the output of create_train_X_y when y=pd.Series(np.arange(10)) and 
    exog is None.
    '''
    forecaster = ForecasterAutoreg(LinearRegression(), lags=5)
    results = forecaster.create_train_X_y(y=pd.Series(np.arange(10)))
    expected = (pd.DataFrame(
                    data = np.array([[4, 3, 2, 1, 0],
                                    [5, 4, 3, 2, 1],
                                    [6, 5, 4, 3, 2],
                                    [7, 6, 5, 4, 3],
                                    [8, 7, 6, 5, 4]]),
                    index   = np.array([5, 6, 7, 8, 9]),
                    columns = ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5']
                ),
                pd.Series(
                    np.array([5, 6, 7, 8, 9]),
                    index = np.array([5, 6, 7, 8, 9]))
               )     

    assert (results[0] == expected[0]).all().all()
    assert (results[1] == expected[1]).all()


def test_create_train_X_y_output_when_y_is_series_10_and_exog_is_series():
    '''
    Test the output of create_train_X_y when y=pd.Series(np.arange(10)) and 
    exog is a pandas series
    '''
    forecaster = ForecasterAutoreg(LinearRegression(), lags=5)
    results = forecaster.create_train_X_y(
                y = pd.Series(np.arange(10)),
                exog =  pd.Series(np.arange(100, 110), name='exog')
              )
    expected = (pd.DataFrame(
                    data = np.array([[4, 3, 2, 1, 0, 105],
                                    [5, 4, 3, 2, 1, 106],
                                    [6, 5, 4, 3, 2, 107],
                                    [7, 6, 5, 4, 3, 108],
                                    [8, 7, 6, 5, 4, 109]]),
                    index   = np.array([5, 6, 7, 8, 9]),
                    columns = ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'exog']
                ),
                pd.Series(
                    np.array([5, 6, 7, 8, 9]),
                    index = np.array([5, 6, 7, 8, 9]))
               )       

    assert (results[0] == expected[0]).all().all()
    assert (results[1] == expected[1]).all()

def test_create_train_X_y_output_when_y_is_series_10_and_exog_is_daraframe():
    '''
    Test the output of create_train_X_y when y=pd.Series(np.arange(10)) and 
    exog is a pandas dataframe with two columns.
    '''
    forecaster = ForecasterAutoreg(LinearRegression(), lags=5)
    results = forecaster.create_train_X_y(
                y = pd.Series(np.arange(10)),
                exog = pd.DataFrame({
                            'exog_1' : np.arange(100, 110),
                            'exog_2' : np.arange(1000, 1010)
                })
              )
        
    expected = (pd.DataFrame(
                    data = np.array([[4, 3, 2, 1, 0, 105, 1005],
                                  [5, 4, 3, 2, 1, 106, 1006],
                                  [6, 5, 4, 3, 2, 107, 1007],
                                  [7, 6, 5, 4, 3, 108, 1008],
                                  [8, 7, 6, 5, 4, 109, 1009]]),
                    index   = np.array([5, 6, 7, 8, 9]),
                    columns = ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'exog_1', 'exog_2']
                ),
                pd.Series(
                    np.array([5, 6, 7, 8, 9]),
                    index = np.array([5, 6, 7, 8, 9])
                )
               )        

    assert (results[0] == expected[0]).all().all()
    assert (results[1] == expected[1]).all()

def test_create_train_X_y_exception_when_y_and_exog_have_different_lenght():
    '''
    Test exception is raised when lenght of y and lenght of exog are different.
    '''
    forecaster = ForecasterAutoreg(LinearRegression(), lags=5)
    with pytest.raises(Exception):
        forecaster.fit(y=pd.Series(np.arange(50)), exog=pd.Series(np.arange(10)))
    with pytest.raises(Exception):
        forecaster.fit(y=pd.Series(np.arange(10)), exog=pd.Series(np.arange(50)))
    with pytest.raises(Exception):
        forecaster.fit(
            y=pd.Series(np.arange(10)),
            exog=pd.DataFrame(np.arange(50).reshape(25,2))
        )
        
def test_create_train_X_y_exception_when_y_and_exog_have_different_index():
    '''
    Test exception is raised when y and exog have diferent index.
    '''
    forecaster = ForecasterAutoreg(LinearRegression(), lags=5)
    with pytest.raises(Exception):
        forecaster.fit(
            y=pd.Series(np.arange(50)),
            exog=pd.Series(np.arange(10), index=np.arange(100, 110))
        )
           
test_create_train_X_y_output_when_y_is_series_10_and_exog_is_None()
test_create_train_X_y_output_when_y_is_series_10_and_exog_is_series()
test_create_train_X_y_output_when_y_is_series_10_and_exog_is_daraframe()
test_create_train_X_y_exception_when_y_and_exog_have_different_lenght()

/home/ximo/Documents/GitHub/skforecast/skforecast/ForecasterBase/ForecasterBase.py:229: UserWarning: `y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.
  '`y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.'
/home/ximo/Documents/GitHub/skforecast/skforecast/ForecasterBase/ForecasterBase.py:317: UserWarning: `exog` has DatetimeIndex index but no frequency. The index is overwritten with a RangeIndex.
  ('`exog` has DatetimeIndex index but no frequency. The index is '
/home/ximo/Documents/GitHub/skforecast/skforecast/ForecasterBase/ForecasterBase.py:229: UserWarning: `y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.
  '`y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.'
/home/ximo/Documents/GitHub/skforecast/skforecast/ForecasterBase/ForecasterBase.py:317: UserWarning: `exog` has DatetimeIndex index but no frequency. The index is overwritten with a RangeIndex.
  ('`exog` has DatetimeIndex index but no freq

In [30]:
# Unit test fit
# ==============================================================================
import pytest
from pytest import approx
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.linear_model import LinearRegression

        
def test_fit_last_window_stored():
    '''
    Test that values of last window are stored after fitting.
    '''    
    forecaster = ForecasterAutoreg(LinearRegression(), lags=3)
    forecaster.fit(y=pd.Series(np.arange(50)))
    expected = pd.Series(np.array([47, 48, 49]), index=[47, 48, 49])
    assert (forecaster.last_window == expected).all()
    
    
def test_fit_in_sample_residuals_stored():
    '''
    Test that values of in_sample_residuals are stored after fitting.
    '''    
    forecaster = ForecasterAutoreg(LinearRegression(), lags=3)
    forecaster.fit(y=pd.Series(np.arange(5)))
    expected = np.array([0, 0])
    results = forecaster.in_sample_residuals  
    assert results.values == approx(expected)
    
test_last_window_stored_when_fit_forecaster()
test_in_sample_residuals_stored_when_fit_forecaster()

/home/ximo/Documents/GitHub/skforecast/skforecast/ForecasterBase/ForecasterBase.py:229: UserWarning: `y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.
  '`y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.'
/home/ximo/Documents/GitHub/skforecast/skforecast/ForecasterBase/ForecasterBase.py:229: UserWarning: `y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.
  '`y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.'
/home/ximo/Documents/GitHub/skforecast/skforecast/ForecasterBase/ForecasterBase.py:229: UserWarning: `y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.
  '`y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.'
/home/ximo/Documents/GitHub/skforecast/skforecast/ForecasterBase/ForecasterBase.py:229: UserWarning: `y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.
  '`y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.'


In [57]:
# Unit test _recursive_predict
# ==============================================================================
import pytest
from pytest import approx
import numpy as np
import pandas as pd
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.linear_model import LinearRegression

def test_predict_output_when_regresor_is_LinearRegression():
    '''
    Test predict output when using LinearRegression as regressor.
    '''
    forecaster = ForecasterAutoreg(LinearRegression(), lags=3)
    forecaster.fit(y=pd.Series(np.arange(50)))
    predictions = forecaster.predict(steps=5)
    expected = pd.Series(
                data = np.array([50., 51., 52., 53., 54.]),
                index = [3, 4, 5, 6, 7]
               )
    assert (predictions.values == approx(expected.values))
    assert (predictions.index == expected.index).all()
    
test_predict_output_when_regresor_is_LinearRegression()

/home/ximo/Documents/GitHub/skforecast/skforecast/ForecasterBase/ForecasterBase.py:229: UserWarning: `y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.
  '`y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.'
/home/ximo/Documents/GitHub/skforecast/skforecast/ForecasterBase/ForecasterBase.py:229: UserWarning: `y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.
  '`y` has no DatetimeIndex index. Index is overwritten with a RangeIndex.'
/home/ximo/Documents/GitHub/skforecast/skforecast/ForecasterBase/ForecasterBase.py:272: UserWarning: `last_window` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex.
  '`last_window` has DatetimeIndex index but no frequency. '


# Información de sesión

In [39]:
import session_info
session_info.show(html=False, write_req_file=False)

-----
matplotlib          3.4.3
numpy               1.19.5
pandas              1.2.5
session_info        1.0.0
skforecast          0.4.dev
sklearn             1.0.1
-----
IPython             7.26.0
jupyter_client      6.1.7
jupyter_core        4.6.3
jupyterlab          2.1.3
notebook            6.4.0
-----
Python 3.7.9 (default, Aug 31 2020, 12:42:55) [GCC 7.3.0]
Linux-5.11.0-38-generic-x86_64-with-debian-bullseye-sid
-----
Session information updated at 2021-10-31 21:47
